# 🏥 Building an Intelligent Medical Assistant with Flotorch Strands

This notebook demonstrates how to build a smart, memory-enabled medical assistant using the `FlotorchStrandsAgent`. Our goal is to create a **Medical Consultation Agent** that intelligently gathers patient information and provides healthcare guidance while remembering patient history across sessions.

### Objective

The agent leverages the `FlotorchStrandsAgent` client to connect to an agent pre-configured on the Flotorch platform. This remote agent is equipped with **Memory capabilities** to perform several key functions:

* **Collect Patient Information**: Gather essential details (Name, Age, Gender)
* **Understand Medical Needs**: Listen to symptoms and health concerns
* **Provide Recommendations**: Offer dietary advice, lifestyle suggestions, and treatment guidance
* **Remember Patient History**: Store and retrieve patient data across sessions using Mem0

### Key Advantage: Agent + Memory Integration

This notebook's primary benefit is the **seamless integration of agent intelligence with persistent memory**. All patient interactions are stored automatically, enabling the agent to:
- Recognize returning patients instantly
- Recall previous consultations and recommendations
- Provide personalized care based on medical history
- Build long-term patient relationships

The agent configuration and memory provider are managed entirely on the Flotorch platform UI, allowing you to update behavior and storage settings without ever changing this notebook's code.

## Prerequisites

Before running this notebook, you must configure two components in the Flotorch console (https://console.flotorch.cloud/):

1. **Agent Configuration**: Create an agent named `medi-assist` (or update the `AGENT_NAME` variable below)
2. **Memory Provider**: Set up a Mem0 memory provider named `memo-provider` (or update the `MEMORY_PROVIDER` variable below)

## Agent + Memory Configuration (Console)

### Step 1: Create Memory Provider

Navigate to: **Console → Memory → Add Memory Provider**

Configure with these values:
```yaml
name: memo-provider
provider: Mem0
scope: Workspace
description: Patient information and consultation history storage
```

### Step 2: Create Agent

Navigate to: **Console → Agents → Add Agent**

Paste this configuration:

```yaml
agent:
  name: medi-assist
  systemPrompt: |
    You are an empathetic medical assistant. You have memory access to store and retrieve patient information.

***IMPORTANT*** : Every Time you need call  this tool FlotorchMemoryTool for every single query to store and retrieve the data.

STEP 1 : IF patient greets you. you need to greet and ask the basic details of the  patient like:
 name, age, and gender.

***IMPORTANT*** : when patient provide their name you need check previous data  with all fields  must should be there if only name ,age , gender is there they are the new patient only one time only you need show full details at the beginning of the basic details. of the that patient based on the app id and user id. if data found show the full details of the patient to the patient and ask for the confirmation like :
IS IT YOU ? : IF patient say yes tell welcome back [name] how are you felling now? and update the data.
IF NOT : new patient flow that steps:

STEP 2: Ask what brings you today here?
STEP 3 : ASK about the additional  symptoms with some examples.
STEP 4 : Ask currently taking any medicines ?

***IMPORTANT *** : 
-1. If Patient using wrong medicine you need correct that and suggest exactly which medicines their need   based on the symptoms and also with dosage. If patient not using any medicines suggest medicines based on the symptoms.
  
- 2. ***Recordation's*** :
 mentions what patient need to Do's and what Don'ts with 3 to 5 lines

STEP 5 : ASK about the past medical history?

STEP 6 : ADD the visit data using [current system date]

AT the end you need store and show to the patient in the structured format like this:

You are an empathetic medical assistant. You have memory access to store and retrieve patient information.

**Your Process:**

1. **First visit:** Ask for name, age, and gender. Then ask "What brings you here today?" Collect symptom details (what, when started, severity, other symptoms).

2. **Current medications:** Ask what medications they're taking. If inappropriate or wrong for their symptoms, suggest correct medications with proper dosage and timing.

3. **Provide recommendations:** Give 2-3 medications with dosage details, dietary advice (3-4 points), lifestyle changes (3-4 points), DO's (4 points), DON'Ts (4 points). Include current visit date.

4. **Store information:** Save complete patient record with all details including visit date.

5. **Returning patients:** When a patient provides their name, check memory. If record exists, show their complete previous information (name, age, gender, symptoms, medications, recommendations, DO's, DON'Ts, last visit date) and ask if they want to continue or update.

6. **Serious symptoms:** If symptoms are severe or concerning, recommend consulting a professional doctor immediately.

Always be warm and empathetic. Never mention memory operations. You are NOT a replacement for a doctor.

**Storage Format:** and **Show Format**:

  "name": "John Doe",
  "age": 28,
  "gender": "Male",
  "symptoms": "Headache, fever",
  "symptom_details": "Started 3 days ago, moderate severity",
  "current_medications": "Paracetamol 500mg",
  "recommended_medications": [
    "Ibuprofen 400mg - Take twice daily after meals",
    "Paracetamol 650mg - If fever exceeds 101°F, max 3 times daily",
     "medical_history": "No prior conditions",
  ],
  "dietary_recommendations": [
    "Drink 8-10 glasses of water daily",
    "Eat light, digestible meals",
    "Include citrus fruits for Vitamin C",
    "Avoid spicy and oily foods"
  ],
  "lifestyle_modifications": [
    "Get 7-8 hours sleep",
    "Avoid stress and screen time",
    "Stay in cool, ventilated room",
    "Apply cold compress on forehead"
  ],
  "dos": [
    "Monitor temperature regularly",
    "Maintain proper hygiene",
    "Take medications on time",
    "Rest adequately"
  ],
  "donts": [
    "Don't skip meals",
    "Avoid self-medication",
    "Don't ignore worsening symptoms",
    "Avoid strenuous activities"
  ],
  "visit_date": [current date based on the system date].

  goal: |
     The medical assistant must conduct empathetic, human-like consultations following this exact flow: (1) Collect name, age, gender intelligently - ask for all three together, then only request missing information without repeating questions. (2) When name is provided, check if returning patient by verifying COMPLETE medical record exists (Name + Age + Gender + Symptoms + Medical_History + Last_Visit) - if only demographics exist without medical data, treat as NEW patient. (3) Ask with empathy "What brings you here today?" and listen to symptoms/concerns. (4) Show empathy ("I'm sorry to hear that") and dig deeper with follow-up questions about duration, severity, triggers. (5) Ask about current medications and VALIDATE if they're appropriate for the condition - correct patient if wrong medication, suggest proper alternatives. (6) Gather medical history and allergies. (7) Provide comprehensive recommendations: suggested medications with dosages, 3-4 detailed lines of dietary advice, 3-4 detailed lines of lifestyle modifications, 4 detailed Do's, 4 detailed Don'ts. (8) Show complete structured summary with all information. (9) Store data SILENTLY without asking patient permission or mentioning "saving/storing". For returning patients, show previous summary, ask "Is this you?", if confirmed continue consultation and update/restore data. Must never repeat questions, never invent data, always validate medications, and maintain empathetic conversational tone.
```

### Step 3: Link Memory to Agent

In the Agent Builder page for `medi-assist`, enable memory and select `memo-provider` as the memory provider.

## 1. Environment Setup

Install the required packages 

In [ ]:
# install flotorch package
%pip install flotorch[strands]==2.4.0b1


## 2. Required Configuration
| Parameter | Description | Example |
|-----------|-------------|----------|
| `FLOTORCH_API_KEY` | Your API authentication key | `sk_...` |
| `FLOTORCH_BASE_URL` | Gateway endpoint | `https://gateway.flotorch.cloud` |
| `AGENT_NAME` | Name of your configured agent | `medi-assist` |
| `MEMORY_PROVIDER` | Memory service name | `memo-provider` |
| `APP_NAME` | Application identifier | `medi-assist_192` |
| `USER_ID` | Unique user identifier | `medi_user_192` |

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
AGENT_NAME = "<your agent name>"
MEMORY_PROVIDER = "<your mem0 provider>"
APP_NAME = "app_name"
USER_ID = "user_id"


## 3. Import Required Libraries

### Purpose
Import all required components for building the medical assistant with Strands framework integration.

### Key Components

#### Strands Framework
- **RepositorySessionManager**: Strands' native session manager that coordinates conversation state and history

#### Flotorch-Strands Integration
- **FlotorchStrandsAgent**: Connects to remotely configured agent on Flotorch platform with Strands orchestration
- **FlotorchStrandsSession**: Custom session repository that stores Strands session data in Flotorch backend
- **FlotorchMemoryTool**: Strands-compatible tool that provides persistent memory capabilities across sessions



In [ ]:
from strands.session.repository_session_manager import RepositorySessionManager
from flotorch.strands.agent import FlotorchStrandsAgent
from flotorch.strands.session import FlotorchStrandsSession
from flotorch.strands.memory import FlotorchMemoryTool

print("✔ Imported necessary libraries successfully")

## 4. Initialize Memory Tool

### Purpose
Configure persistent long-term memory as a Strands tool for cross-session patient data storage.

### What is FlotorchMemoryTool?
A Strands-compatible tool that wraps Flotorch's memory service, exposing memory operations (add, search, retrieve) as callable tool functions within the Strands agent framework.

### Configuration Parameters
- **api_key**: Flotorch API authentication key
- **base_url**: Flotorch gateway endpoint
- **provider_name**: Name of the memory provider configured in Flotorch console (e.g., `memo-provider`)
- **user_id**: Unique user identifier for memory isolation
- **app_id**: Application identifier for memory scoping

### Tool Integration
- **Tools List**: Memory tool is added to the agent's toolset for direct invocation
- **Agent Access**: The agent can call memory operations using natural language
- **Automatic Storage**: Patient data is stored when the agent invokes the memory tool

In [ ]:
memory = FlotorchMemoryTool(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    provider_name=MEMORY_PROVIDER,
    user_id=USER_ID,
    app_id=APP_NAME
)

tools = [memory]

print(f"✔ memory initialized.")

## 6. Initialize Session Repository & Manager

### Purpose
Set up Strands session management infrastructure with Flotorch backend for conversation state persistence.

### What is FlotorchStrandsSession?
A custom Strands session repository that implements the Strands `SessionRepository` interface, storing all session data (conversation history, agent state, messages) in Flotorch's backend instead of local storage.

### Configuration Components

#### FlotorchStrandsSession (Repository)
- **api_key**: Flotorch API authentication key
- **base_url**: Flotorch gateway endpoint
- Acts as the storage layer for Strands sessions

#### RepositorySessionManager
- **session_id**: Unique identifier for the conversation session (user-provided or generated)
- **session_repository**: The FlotorchStrandsSession instance for data persistence
- Strands' native manager that coordinates session operations

In [ ]:

repository = FlotorchStrandsSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
)

In [ ]:
session_id = input("Enter session ID (or press Enter for new session): ").strip()

session_manager = RepositorySessionManager(
    session_id=session_id,
    session_repository=repository
)

print(f"✔ Session storage initialized.")

## 7. Initialize Medical Agent

### Purpose
Connect to the remotely configured medical assistant agent and integrate it with Strands session manager and memory tools.

### Configuration
Initializes `FlotorchStrandsAgent` with:
- **agent_name**: Name of the agent configured in Flotorch console
- **custom_tools**: List of Strands tools including FlotorchMemoryTool
- **session_manager**: RepositorySessionManager for conversation state management

### Key Features
- **Remote Configuration**: Loads agent settings (system prompt, model) from Flotorch platform
- **Tool Integration**: Memory tool available for agent to invoke during conversations
- **Session Management**: Integrates with Strands' session manager for state persistence
- **Automatic Sync**: Periodically reloads agent configuration from Flotorch


In [ ]:
client = FlotorchStrandsAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools=tools,
    session_manager=session_manager,  # ✅ ENABLED - Required for tool execution and conversation state
)

agent = client.get_agent()

print("✔ Initialized FlotorchStrandsAgent with custom tools and memory")

## 8. Interactive Chat Interface

### Purpose
Start an interactive conversation with the medical assistant.

### Features
- **Continuous Conversation**: Chat until you type "exit"
- **Automatic Memory**: Patient data saved automatically
- **Session Persistence**: Conversation context maintained throughout

### Usage Instructions
1. Run the cell below
2. Enter your messages when prompted
3. Type `exit` to end the conversation

### Example Interaction Flow
```
You: Hello
Dr. Hello! How can I assist you with your health today? Before we get started, may I have your name, age, and gender?

You: My name is John, I'm 35, male
Dr. AI: Thank you, John. What brings you here today?

You: I have a headache
Dr. AI: I understand. Can you describe the headache? When did it start?...
```

### Behind the Scenes
- Agent collects: Name, Age, Gender
- Then gathers: Symptoms, Medical History, Medications, Allergies
- Stores everything in structured memory format
- Next visit: Automatically recognizes "John" and retrieves history

In [ ]:
while True:
    user_query = input("You: ")
    if user_query.lower().strip() == "exit":
        break

    try:
        response = agent(user_query)
        print(f"Assistant: {response}\n")
    except Exception as e:
        print(f"Error: {e}\n")

---
## Summary

### What We Built

A production-ready medical assistant that:

### Recognizes Returning Patients
- Automatically identifies patients by name
- Retrieves complete medical history
- Shows summary and asks for confirmation

### Persistent Memory
- Stores patient records across sessions
- Tracks symptoms, diagnoses, and recommendations
- Maintains visit history with timestamps

### Professional Workflow
- Collects demographics first (Name, Age, Gender)
- Asks symptom-specific diagnostic questions
- Provides recommendations and doctor referrals when needed

### Key Benefits

#### Zero Code Maintenance
- Agent configured remotely in Flotorch Console
- Update prompts without touching code
- Memory provider managed centrally

#### Seamless Patient Experience
- Patients recognized automatically
- No manual data entry required
- Continuous care across visits

### Data Storage Format

Patient information is automatically stored as:
```json
{
  "Name": "John Doe",
  "Age": 35,
  "Gender": "Male",
  "Symptoms": "Headache, fatigue",
  "Diagnosis": "Tension headache",
  "Medical History": "No prior conditions",
  "Current Medications": "None",
  "Allergies": "None known",
  "Dietary Recommendations": "Stay hydrated, reduce caffeine",
  "Lifestyle Advice": "Regular sleep schedule, stress management",
  "Last Visit": "2025-10-30"
}
```

### Next Steps

1. **Customize**: Update agent behavior in Console → Agents → medi-assist
2. **Scale**: Add more patients and sessions as needed
3. **Monitor**: Check logs in Console → Logs tab

---

## 📚 Resources

- [Flotorch Documentation](https://docs.flotorch.cloud)
- [Agent Configuration Guide](https://console.flotorch.cloud/docs/agents)
- [Memory Service API](https://console.flotorch.cloud/docs/memory)